[Reference](https://medium.com/towards-artificial-intelligence/using-twitter-rest-apis-in-python-to-search-and-download-tweets-in-bulk-da234b5f155a)

In [2]:
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()
%matplotlib inline

In [3]:
config = {"twitterConsumerKey":"XXXX", "twitterConsumerSecretKey" :"XXXX"} 
auth = tweepy.OAuthHandler(config["twitterConsumerKey"], config["twitterConsumerSecretKey"]) 
redirect_url = auth.get_authorization_url() 
redirect_url

In [4]:
pin = "XXXX"
auth.get_access_token(pin)

# Searching for tweets

In [5]:
query = 'British Airways'
max_tweets = 10
searched_tweets = [status for status in tweepy.Cursor(api.search, q=query,tweet_mode='extended').items(max_tweets)]

search_dict = {"text": [], "author": [], "created_date": []}

for item in searched_tweets:
    if not item.retweet or "RT" not in item.full_text:
        search_dict["text"].append(item.full_text)
        search_dict["author"].append(item.author.name)
        search_dict["created_date"].append(item.created_at)
\df = pd.DataFrame.from_dict(search_dict)
df.head()

# Language detection

In [6]:
from langid.langid import LanguageIdentifier, model
def get_lang(document):
    identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
    prob_tuple = identifier.classify(document)
    return prob_tuple[0]
    
df["language"] = df["text"].apply(get_lang)

In [7]:
print(df["language"].unique())
df_filtered = df[df["language"]=="en"]
print(df_filtered.shape)

# Getting sentiments to score for tweets

In [8]:
from textblob import TextBlob

def get_sentiments(text):
    blob = TextBlob(text)
#     sent_dict = {}
#     sent_dict["polarity"] = blob.sentiment.polarity
#     sent_dict["subjectivity"] = blob.sentiment.subjectivity
    
    if blob.sentiment.polarity > 0.1:
        return 'positive'
    elif blob.sentiment.polarity < -0.1:
        return 'negative'
    else:
        return 'neutral'def get_sentiments_score(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity
    
df_filtered["sentiments"]=df_filtered["text"].apply(get_sentiments)
df_filtered["sentiments_score"]=df_filtered["text"].apply(get_sentiments_score)
df_filtered.head()